# 自動更新景點標籤 ( from 評論 ) :
# 一天最多更新一次

# 評論倒出資料庫 -> 評論撈出關鍵字 -> 
# 評論關鍵字分類 -> 評論貼完標籤 -> 輸出成JSON檔

In [1]:
####################################################################

#[1]
####################################################################
def DumpComment(IP,User,PWd,DB,sql,Folder,PrintReport):
    import MySQLdb
    import os
    import re
    db = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫
    cursor = db.cursor()

    count=0#　計算讀取的資料量
    CntDic = {}#　Key:景點名稱　Value:評論數
    Tmp = {}#　Key:景點編號　Value:景點名稱
    os.popen('mkdir '+Folder)#　新增資料夾
    try:
        cursor.execute(sql)
        results = cursor.fetchall()

        for row in results: 
            if str(row[2]) in Tmp:#　如果已在Set裡           
                CntDic[row[0].encode('utf-8')] += 1
                with open(Folder+'\\'+str(row[2])+'.txt','a') as f:
                    f.write(row[1].encode('utf-8')+'\n')#　就繼續寫下去

            else:#　如果還不在Tmp裡
                Tmp[str(row[2])] = row[0].encode('utf-8')#　寫Tmp(Key:景點編號 Value:景點名稱)
                CntDic[row[0].encode('utf-8')] = 1
                with open(Folder+'\\'+str(row[2])+'.txt','w') as f2:
                    f2.write('\n'+row[0].encode('utf-8')+'\n'\
                                    +row[1].encode('utf-8')+'\n')#　把Title寫在第二行!
            count += 1  

        S1 = set() #存放全部'景點名稱'
        S2 = set() #存放'重複的景點名稱'  
        for a in Tmp:#　遍歷第一次Tmp 把重複的寫進S2的Set裡面
            if Tmp[a] not in S1:#　如果是第一次讀到,則寫進S1裡面
                S1.add(Tmp[a])
            else:#　如果已經在S1裡(非首次讀到此"景點名稱",表出現重複景點名稱,加進S2裡)
                S2.add(Tmp[a])

        Duplicate = 0
        for b in Tmp:#　遍歷第二次Tmp 
            if Tmp[b] in S2:#　如果已經在S2裡(表示出現重複的,則印出錯誤訊息!)
                Duplicate += 1 #　代表出現重複
                print b,Tmp[b],'  Duplicate!'#　印出重複訊息

        if Duplicate >= 1:
            #print 'Attraction Duplicate! Please Check!'
            print '\n\nDuplicate!!!!!!!!!!!!!!!!\n\n'
            ##### 刪掉連資源回收桶都找不到  慎用!!! #####
            #也可以選擇 : 若出現重複...把全部txt檔都刪掉
            #####command = 'cd {} & del *.txt'
            #####os.popen(command.format(Folder))
            ##########################################

    except KeyError:
        print "Error: unable to fecth data"

    if PrintReport == 'Y':
        print "Num of Queries:",str(count)
        print 'Num of Attractions:',len(CntDic)
        Cnt = 0
        for i in CntDic:
            Cnt += CntDic[i]
            #####
            #可決定是否要印出計算字典
            #print i,str(CntDic[i]).decode('string_escape')
            #####
        print 'Num of Comments:',Cnt
        if Cnt == count:
            print 'Dump Comments Successfully!'
####################################################################




#[2]
####################################################################
def wordsToList(attfile):
    import re
    f = open(attfile,'r')
    f1 = f.read()
    f2 = ''.join(f1.split())
    words=jieba.cut(f2)
    
    li = []
    for u in words: #把"有出現兩個連續英數字"的詞語濾掉
        if not re.search('(\w)(\w)',u):
            li.append(u)
    n1 = (' '.join(li)).encode('utf-8')
    return n1
    f.close()

def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip().decode('utf-8'))#要解開utf-8,才能成功給scikitlearn用
    fid.close()
    return li

def SciNoEng(date,listInput,stopWordsFile,rankNum,title):
    from sklearn import feature_extraction  
    from sklearn.feature_extraction.text import TfidfTransformer  
    from sklearn.feature_extraction.text import CountVectorizer
    filename = 'NE_SciTfidf-'+str(rankNum)+'-'+date+'.txt'
    
    #創造一個空矩陣
    corpus = []
    
    #把傳入的List依序讀出，使用方法把這些東西全都讀出來
    for each in range(0,len(listInput)):
        corpus.append(wordsToList(listInput[each]))

    vectorizer=CountVectorizer(analyzer='word',stop_words=stopWordsList(stopWordsFile))  #該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    x = vectorizer.fit_transform(corpus)  #將文本轉為詞頻矩陣 
    transformer=TfidfTransformer()  #該類會統計每個詞語的tf-idf權值 
    tfidf=transformer.fit_transform(x)  #計算tf-idf
    word=vectorizer.get_feature_names()  #獲取詞袋模型中的所有詞語
    weight=tfidf.toarray()   #將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  
    
    with open(filename,'a') as files:
        import operator
        for i in range(len(weight)):
            
            #*#*#*#*#*#*#*#*#*#
            #決定是否要印景點標題#
            
            #print listInput[i]
            #files.write(listInput[i])
            #files.write('\n')
            
            TitleDic=['True','T','t','Yes','Y','y']
            if title in TitleDic:
                files.write(listInput[i])
                #print listInput[i]
                files.write('\n')
            else:
                files.write('')
            
            #*#*#*#*#*#*#*#*#*#
            dic = {}
            for j in range(len(word)):  
                dic[word[j]] = weight[i][j]
            words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
            num2 = 0
            for ele in words_freq[0:rankNum]:
                num2 += 1
                if not ele[1] == float(0.0):
                    #print num2,"\t",ele[0],"\t",ele[1]
                    files.write(ele[0].encode('utf-8'))
                    files.write("\n")
                    
def ExtractKeyWords(Folder,Rank,StopWords,PrintReport):
    import time
    import os
    x = time.time()
    lis = []
    for name in os.listdir(Folder):
        #path = Folder+'\\'+name
        lis.append(Folder+'\\'+name)
        
    Date = time.strftime('%Y%m%d',time.localtime(time.time()))
    
    SciNoEng(Date,lis,StopWords,Rank,'Y')
    y = time.time()
    if PrintReport == 'Y':
        print 'Extract Keywords Successfully!'
        print 'Cost ',y-x,' secs.\n'
        
####################################################################




#[3]
####################################################################
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model):
    
    import gensim
    #1__WIKI
    if Model == 'Wiki':
        model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")
    #2__搜狐新闻数据
    elif Model == 'SogouCS':
        model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')
    #3__全网新闻数据
    elif Model == 'SogouCA':
        model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')
       
    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理
            
#####　　把分類結果字典回傳　　#####
    return ClassingResult

##################################################################################################

##################################################################################################

def AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,\
                     Concept,Corpus,SimLevel,SimPercent,Model):
    
    import re
    from pprint import pprint
    total = 0#　計算總共有多少行資料
    title = 0#　計算有多少的景點
    length = 0#　計算有多少個關鍵字

    ReAttr = '('+Folder+')(.)(.*)'#　計算表頭，正規化用
    for i in open(KeyWordsFile,'r'):#　打開結果的檔案 
        if re.match(ReAttr,i.strip()):#　計算表頭(來源的資料夾)
            title += 1#　表頭(景點)＋１
        else:#　不是表頭。就是關鍵字
            length += 1#　關鍵字＋１
        total += 1#　全部＋１

    if PrintReport == 'Y':
        print "景點總數:",title," 關鍵字總數:",length," 原始檔長度:",total#　把數字都印出來
        if (title+length) == total:#　如果 (景點+關鍵字) = (總行數)
            print '1. 檔案讀取成功! 並無遺漏!\n'#　代表沒有遺漏！成功讀取
        else:
            print '1. 錯誤! 請檢查!\n'

    domain = Folder+'\\'#  景點評論所在的資料夾(同時也是分割內容的標準)

    with open(KeyWordsFile,'r') as fid:#　再讀一次結果的檔案
        content = fid.read()#　把內容讀出來，存給content 
        DictA = {}#　創建一字典
        c = 0#　另一變數c 初值為0

        for i in content.split(domain):#　把內容依據CommentSample(存放景點評論的資料夾名稱)切割
            if c > 0:#　因為split的結果，第0個(也就是for迴圈第一次進來)的值會是空的，所以至少要第二次進來的我才要
                d = 0#　另一變數d 初值為0
                Now = ''

                for o in i.split():#　用空白把每個資料切開
                    if d == 0:#　split的結果，第0個會是檔名(EX: 15.txt)，並不是關鍵字，要特別處理
                        Now = o.strip()#　如果是表頭的話，就把文字指定給外面的變數Now
                        DictA[Now] = []#　把表頭新增進字典裡，並指定內容為一陣列
                    else:#　關鍵字都會進來這裡
                        DictA[Now].append(o.strip())#　把關鍵字加進字典的Value陣列裡
                    d+=1#　做完之後，把d加一。目的僅是因為首次進來(d=0)時，要為字典新增KEY
                        #  以便之後的關鍵字有家可歸，之後繼續加上去並無實值意義
            c+=1#　做完之後，把c加一。目的僅是為了避免第一次進來(c=0)的情況，之後繼續加上去並無實值意義

    ######　　做完以上動作，會得到一字典DictA，KEY:(各景點編號.txt) Value:(各景點的關鍵字)

    DictB = {}#  創建一字典(目的是要把DictA的KEY)轉換成中文
    cnt = 0#  計算關鍵字數量用

    for a in DictA:#  把字典的KEY(EX: 15.txt)讀出來
        count = 0#  另一變數初值為 0
        for u in open(domain+a,'r'):#  (資料夾+字典的KEY) = 檔案的相對位置；把檔讀出來 
            count+=1#  每次進來都先+1，表是讀到第 N 行
            if count == 2:#  當count = 2(讀到第二行, 也就是景點名稱)時，(假設讀到15.txt, 第二行是'陽明山')
                DictB[u.strip()] = DictA[a]#  就把(15.txt)的Value指定給新的字典的'陽明山'這個KEY，達到檔名轉中文的目的
                break#  得到中文景點名稱之後，就沒有必要讀下去，直接中斷此次迴圈
        for b in DictA[a]:#  把每個KEY的關鍵字讀出來
            cnt += 1#  計算關鍵字數量

        #####印出原始字典(DictA)#####
        if PrintDict == 'PrintA':
            print a,'\n',str(DictA[a]).decode('string_escape')
        ###########################

    if PrintReport == 'Y':
        print 'DictA內含詞語總量: ',cnt
        if cnt == length:
            print '2. DictA寫入成功! 並無遺漏!\n'
        else:
            print '2. 錯誤! DictA寫入過程出現遺漏!\n'


    cnt2 = 0#  計算關鍵字數量用
    #####印出最終字典#####
    for g in DictB:
        if PrintDict == 'PrintB':
            print g,'\n',str(DictB[g]).decode('string_escape') 
    #####################
        for r in DictB[g]:
            cnt2 += 1

    if PrintReport == 'Y':
        print '\nDictB內含詞語總量: ',cnt2
        if cnt2 == cnt:
            print '3. DictB寫入(轉換)成功! 並無遺漏!\n'
        else:
            print '3. 錯誤! DictB寫入(轉換)過程出現遺漏!\n'  
            
            
###***************************************************
    DictFinal = DictB
    ClassingResult=W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model)    
###***************************************************

    if PrintDict == 'PrintConcept':
        for ee in ClassingResult:
            print ee,str(ClassingResult[ee]).decode('string_escape')
    
    
    DictResult={}#　把景點關鍵字的結果轉成'概念'.'文字雲'
    for a in DictFinal:#　把{景點:[關鍵字]}的字典讀出來
        DictResult[a] = {'概　念':{},'文字雲':[]}#　指定KEY:景點 Value:概念{} 文字雲

        for b in DictFinal[a]:#　把每個景點的關鍵字讀出來
            z = 'Homeless'#　先假設每個關鍵字都沒有家

            for c in ClassingResult:#　讀取每個概念,內容是{概念:{詞語:比重}}讀出來
                if b in ClassingResult[c]:#　如果b(現在讀到的這個關鍵字)在某概念裡面
                    z = 'Home'#　有家了

                    for e in ClassingResult[c]:#　把概念的詞都讀一讀
                        if b == e:#　如果'概念內的詞'='現在的關鍵字'-->抓到了!
                            if not c in DictResult[a]['概　念']:#　如果景點還沒有此概念
                                DictResult[a]['概　念'][c]={}#　為景點的此概念新增一個字典
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]# 並且把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度    
                            else:#　如果景點已經有此概念了
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]#　把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度
                            break#　因為以經分好家了,寫完之後就跳出迴圈
                    break#　因為已分好家,所以再次跳出迴圈

            if z == 'Homeless':#　如果字詞並沒有屬於任何概念,依然沒有家
                DictResult[a]['文字雲'].append(b)#　就把字詞寫進該景點的'文字雲'裡面
                
                
    #驗證結果
    if PrintReport == 'Y':
        Class = 0
        Cloud = 0
        for g in DictResult:
            for h in DictResult[g]:
                if h == '文字雲':
                    Cloud += len(DictResult[g][h])
                if h == '概　念':
                    for k in DictResult[g][h]:
                        Class += len(DictResult[g][h][k])
                        
        print 'DictReSult內含詞語總量: ',(Class + Cloud)
        if (Class + Cloud) == cnt2:
            print '4. DictReSult寫入(轉換)成功! 並無遺漏!\n'
        else:
            print '4. 錯誤! DictReSult寫入(轉換)過程出現遺漏!\n'


    if PrintDict == 'PrintC':
        for g in DictResult:
            print '【',g,'】',
            for u in DictResult[g]['概　念']:
                print u,
            print '\n'        
            
    if PrintDict == 'PrintD':
        for g in DictResult:
            print '【',g,'】'
            for h in DictResult[g]:
                print '｢',h,'｣'

                if h == '概　念':
                    for k in DictResult[g][h]:
                        print k,len(DictResult[g][h][k]),str(DictResult[g][h][k]).decode('string_escape')
                elif h == '文字雲':
                    print str(DictResult[g][h]).decode('string_escape')
            print ''

    ###決定是否回傳字典
    
    if ReturnDict == 'ReturnA':
        return DictA
    elif ReturnDict == 'ReturnB':
        return DictB
    elif ReturnDict == 'ReturnC':
        return DictResult
    elif ReturnDict == 'ReturnConcept':
        return ClassingResult
    
####################################################################




#[4]
####################################################################    
    
def DumpWC(Result,WordCloudFile):
    import json
    x = 0
    y = 0
    WordCloud = {}
    for a in Result:#　讀取結果的各個景點
        for b in Result[a]:#　讀取各個景點的Value
            if b == '文字雲':
                for c in Result[a][b]:#　讀取文字雲的內容(陣列)
                    if not c in WordCloud:#　如果是第一次讀到
                        WordCloud[c] = 1#　則加進字典, 計算次數為一
                    else:#　如果已經讀過了
                        WordCloud[c] += 1#　則字典該詞語次數加一
                x += 1#　計算景點數
                y += len(Result[a][b])#　計算文字雲的字彙量

    z = 0
    for e in WordCloud:
        z += WordCloud[e]#　計算文字雲的字彙量

    if z==y:
        j_str = json.dumps(WordCloud)#　丟進一JSON格式的變數
        with open(WordCloudFile,'w') as f:#　寫進Json檔裡
            f.write(j_str)
        print 'Attr :',x,'Words :',len(WordCloud),'TotalWords :',z
        print 'Trans To WordCloud Dict Successfully!'

# =====================================

# 概念字典

In [2]:
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

# 設定參數

In [3]:
# Step 1 　從資料庫把東西倒出來
# [一] #
###　　　資料庫連結資訊　　　###
IP   = "10.120.26.4"
User = "iii"
PWd  = "iii"
DB   = "iii"
###　　　###　　　###　　　####
# [二] #
######　　　SQL命令　　　######
sql = "SELECT a.Attr_name, c.C_content, a.Attr_id\
       FROM  `iiimap_attraction` AS a\
       JOIN  `iiimap_comment` AS c ON a.Attr_id = c.Attr_id\
       ORDER BY a.Attr_name ASC;"
###　　　###　　　###　　　####
#############################################################


# Step 2 　淬取關鍵字
# [一] #
######　　決定淬取排名　　######
Rank = 30
StopWords = 'Stop_Edit_Final_20160114.txt'
# [二] #
######　　　讀取字典　　　######
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》
jieba.load_userdict('dict_Custom_20160114.txt') # 自定義字典
jieba.analyse.set_stop_words("Stop_Edit_Final_20160114.txt") #停用詞
#############################################################


# Step 3 　更新景點標籤
import time
Date = time.strftime('%Y%m%d',time.localtime(time.time()))
# [一] #
######　　　輸出相關參數　　　######
KeyWordsFile = 'NE_SciTfidf-'+str(Rank)+'-'+Date+'.txt'
PrintDict = 'N'
ReturnDict = 'ReturnC'
# [二] #
######　　　W2V相關參數　　　######
Concept   = Dic
Corpus    = 'NE_SciTfidf-'+str(Rank)+'-'+Date+'.txt'
SimLevel  = 0.35
SimPercent= 0.49
Model     = 'Wiki'
#############################################################


# Step 4 　決定重要資訊
#　　要放置評論的資料夾
Folder = 'Comment_'+Date
#　　印出報告
#Step 1 - 2
PrintReport = 'Y'
#Step 3 - 4
PrintReport2 = 'N'
#　　JSON檔案命名
JsonFile = 'CommentTags-'+Date+'.json'#　　　　標籤結果
WordCloudFile = 'WordCloud-'+Date+'.json'#　　 文字雲檔案
#############################################################

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.270 seconds.
DEBUG:jieba:Loading model cost 0.270 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


# 使用方法 --> 一次更新
需要檔案：

    一、字典、停用詞
    二、Word2Vec模型
    
使用結果：

    產生下列資料：
       1.'Comment_...'[資料夾]：景點的所有評論
       2.'NE_SciTfidf-'[TXT檔]：景點的所有關鍵字
       3.'CommentTags-'[json檔]：景點的標籤結果
       4.'WordCloud-..'[json檔]：文字雲數據(KEY:文字 VALUE:字頻)

In [4]:
import time
a = time.time()
print 'Start Update Tags...\n'

print 'Step1 Dump Comments...'
DumpComment(IP,User,PWd,DB,sql,Folder,PrintReport)
print 'Step1 Complete!\n'

print 'Strp2 Extract Keywords...'
ExtractKeyWords(Folder,Rank,StopWords,PrintReport)
print 'Step2 Complete!\n'

print 'Strp3 Update Attraction Tags...'
Result = AttrClassify(KeyWordsFile,Folder,PrintReport2,PrintDict,ReturnDict,Concept,Corpus,SimLevel,SimPercent,Model)
print 'Step3 Complete!\n'

print 'Strp4 Dump Taggin Result Into Json Files...'
import json
json_str = json.dumps(Result)#　丟進一JSON格式的變數
with open(JsonFile,'w') as jfid:#　寫進Json檔裡
    jfid.write(json_str)
print 'Step4 Complete!\n'

print 'Strp5 Dump WordCloud Result Into Json Files...'
DumpWC(Result,WordCloudFile)
print 'Step5 Complete!\n'

b = time.time()
print 'All Complete! Cost',str(b-a),'secs.'

Start Update Tags...

Step1 Dump Comments...
Num of Queries: 52660
Num of Attractions: 226
Num of Comments: 52660
Dump Comments Successfully!
Step1 Complete!

Strp2 Extract Keywords...
Extract Keywords Successfully!
Cost  23.9499998093  secs.

Step2 Complete!

Strp3 Update Attraction Tags...
Step3 Complete!

Strp4 Dump Taggin Result Into Json Files...
Step4 Complete!

Strp5 Dump WordCloud Result Into Json Files...
Attr : 226 Words : 2905 TotalWords : 3715
Trans To WordCloud Dict Successfully!
Step5 Complete!

All Complete! Cost 125.044000149 secs.


# =====================================

# 也可以把字典輸出成PICKLE檔，以供別的方法使用

In [15]:
# Export
import pickle
PickleFile = 'Pickle_0121'
with open(PickleFile+'.p',"wb") as f:
    pickle.dump(Result, f)

In [8]:
# Import
import pickle
with open(PickleFile+'.p', 'rb') as f2:
    data = pickle.load(f2)
print type(data)

<type 'dict'>
